# Introduction to `fastplotlib` 🚀

This notebook goes through the basic components of the `fastplotlib` API, image, line, scatter plots, subplots and simple animations

**The example images are from `imageio` so you will need to install it for this example notebook. But `imageio` is not required to use `fasptlotlib`**

In [ ]:
!pip install imageio

In [ ]:
import imageio.v3 as iio

In [ ]:
import fastplotlib as fpl
from ipywidgets import VBox, HBox, IntSlider
import numpy as np

In [ ]:
# this is only for testing, you do not need this to use fastplotlib
from nb_test_utils import plot_test, notebook_finished, TESTING

## Simple image

We are going to be using `jupyterlab-sidecar` to render some of the plots on the side. This makes it very easy to interact with your plots without having to constantly scroll up and down :D

In [ ]:
# create a `Figure` instance
# by default the figure will have 1 subplot
fig = fpl.Figure()

# get a grayscale image
data = iio.imread("imageio:camera.png")

# plot the image data
image_graphic = fig[0, 0].add_image(data=data, name="sample-image")

# show the plot
fig.show(sidecar=True)

**Use the handle on the bottom right corner of the _canvas_ to resize it. You can also pan and zoom using your mouse!**

If an image is in the plot the origin is in the top left. You can click the flip button to flip the y-axis direction, or use `plot.camera.local.scale_y *= -1`

Changing graphic **"features"**

In [ ]:
image_graphic.cmap = "viridis"

### Slicing data

**Most features, such as `data` support slicing!**

Out image data is of shape [n_rows, n_cols]

In [ ]:
image_graphic.data().shape

In [ ]:
image_graphic.data[::15, :] = 1
image_graphic.data[:, ::15] = 1

**Fancy indexing**

In [ ]:
image_graphic.data[data > 175] = 255

Adjust vmin vmax

In [ ]:
image_graphic.cmap.vmin = 50
image_graphic.cmap.vmax = 150

In [ ]:
# testing cell, ignore
plot_test("camera", fig)

**Set the entire data array again**

Note: The shape of the new data array must match the current data shown in the Graphic.

In [ ]:
new_data = iio.imread("imageio:astronaut.png")
new_data.shape

This is an RGB image, convert to grayscale to maintain the shape of (512, 512)

In [ ]:
gray = new_data.dot([0.3, 0.6, 0.1])
gray.shape

In [ ]:
image_graphic.data = gray



reset vmin vmax

In [ ]:
image_graphic.cmap.reset_vmin_vmax()

In [ ]:
# testing cell, ignore
plot_test("astronaut", fig)

In [ ]:
fig.canvas.get_logical_size()

### Indexing subplots

**Subplots are indexable and give you their graphics by name**

In [ ]:
fig[0, 0]

Access graphics in a subplot

In [ ]:
# by name
fig[0, 0]["sample-image"]

In [ ]:
# or through the .graphics property of a subplot
fig[0, 0].graphics

In [ ]:
# these are the same!
fig[0, 0].graphics[0] is fig[0, 0]["sample-image"]

The `Graphic` instance is also returned when you call `plot.add_<graphic_type>`.

In [ ]:
image_graphic

In [ ]:
image_graphic == fig[0, 0]["sample-image"]

In [ ]:
# close the figure
fig.close()

### RGB images are also supported

`cmap` arguments are ignored for rgb images, but vmin vmax still works

In [ ]:
fig_rgb = fpl.Figure()

fig_rgb[0, 0].add_image(new_data, name="rgb-image")

# show the figure
fig_rgb.show()

vmin and vmax are still applicable to rgb images

In [ ]:
fig_rgb[0, 0]["rgb-image"].cmap.vmin = 100

In [ ]:
# testing cell, ignore
plot_test("astronaut_RGB", fig_rgb)

In [ ]:
# close figure
fig_rgb.close()

### Image updates

This examples show how you can define animation functions that run on every render cycle.

In [ ]:
# create a figure
fig_v = fpl.Figure()

fig.canvas.max_buffered_frames = 1

# make some random data again
data = np.random.rand(512, 512)

# plot the data
fig_v[0, 0].add_image(data=data, name="random-image")

# a function to update the image_graphic
# a figure-level animation function will optionally take the figure as an argument
def update_data(figure_instance):
    new_data = np.random.rand(512, 512)
    figure_instance[0, 0]["random-image"].data = new_data

# you can also add animation functions to individual subplots
def update_data_subplot(subplot_instance):
    pass

# add this as an animation function
fig_v.add_animations(update_data)

# similarly you can add animation function to a subplot
# fig_v[0, 0].add_animations(update_data_subplot)

# show the plot
fig_v.show()

### We can share controllers across plots

This example creates a new figure, but it share the pan-zoom controllers from the previous figure!

In [ ]:
fig_sync = fpl.Figure(controllers=fig_v.controllers)

data = np.random.rand(512, 512)

image_graphic_instance = fig_sync[0, 0].add_image(data=data, cmap="viridis")

# you will need to define a new animation function for this graphic
def update_data_2():
    new_data = np.random.rand(512, 512)
    # alternatively, you can use the stored reference to the graphic as well instead of indexing the subplot
    image_graphic_instance.data = new_data

fig_sync.add_animations(update_data_2)

fig_sync.show()

#### Keeping a reference to the Graphic instance, as shown above `image_graphic_instance`, is useful if you're creating something where you need flexibility in the naming of the graphics

### You can also use `ipywidgets.VBox` and `HBox` to stack plots. See the `subplot` notebooks for more automated subplotting

In [ ]:
HBox([fig_v.show(), fig_sync.show()])

In [ ]:
# close figures
fig_v.close()
fig_sync.close()

# Line plots

## 2D line plots

This example plots a sine wave, cosine wave, and ricker wavelet and demonstrates how **Graphic Features** can be modified by slicing!

### First generate some data.

In [ ]:
# linspace, create 100 evenly spaced x values from -10 to 10
xs = np.linspace(-10, 10, 100)
# sine wave
ys = np.sin(xs)
sine = np.column_stack([xs, ys])

# cosine wave
ys = np.cos(xs) + 5
cosine = np.column_stack([xs, ys])

# sinc function
a = 0.5
ys = np.sinc(xs) * 3 + 8
sinc = np.column_stack([xs, ys])

### We will plot all of it on the same plot. Each line plot will be an individual Graphic, you can have any combination of graphics on a plot.

In [ ]:
# Create a figure
fig_lines = fpl.Figure()

# we will add all the lines to the same subplot
subplot = fig_lines[0, 0]

# plot sine wave, use a single color
sine_graphic = subplot.add_line(data=sine, thickness=5, colors="magenta")

# you can also use colormaps for lines!
cosine_graphic = subplot.add_line(data=cosine, thickness=12, cmap="autumn")

# or a list of colors for each datapoint
colors = ["r"] * 25 + ["purple"] * 25 + ["y"] * 25 + ["b"] * 25
sinc_graphic = subplot.add_line(data=sinc, thickness=5, colors = colors)

# show the plot
fig_lines.show(sidecar=True, sidecar_kwargs={"title": "lines"})

In [ ]:
# testing cell, ignore
plot_test("lines", fig_lines)

### "stretching" the camera, useful for large timeseries data

Set `maintain_aspect = False` on a camera, and then use the right mouse button and move the mouse to stretch and squeeze the view!

You can also click the **`1:1`** button to toggle this, or use `subplot.camera.maintain_aspect`

### reset the plot area

In [ ]:
subplot.auto_scale(maintain_aspect=True)

## Graphic features support slicing! :D 

In [ ]:
# indexing of colors
cosine_graphic.colors[:15] = "magenta"
cosine_graphic.colors[90:] = "red"
cosine_graphic.colors[60] = "w"

# indexing to assign colormaps to entire lines or segments
sinc_graphic.cmap[10:50] = "gray"
sine_graphic.cmap = "seismic"

# more complex indexing, set the blue value directly from an array
cosine_graphic.colors[65:90, 0] = np.linspace(0, 1, 90-65)

## You can capture changes to a graphic feature as events

In [ ]:
def callback_func(event_data):
    print(event_data)

# Will print event data when the color changes
cosine_graphic.colors.add_event_handler(callback_func)

In [ ]:
# more complex indexing of colors
# from point 15 - 30, set every 3rd point as "cyan"
cosine_graphic.colors[15:50:3] = "cyan"

In [ ]:
# testing cell, ignore
plot_test("lines-colors", fig_lines)

## Graphic _data_ is also slicable and settable

In [ ]:
cosine_graphic.data[10:50:5, :2] = sine[10:50:5]
cosine_graphic.data[90:, 1] = 7

In [ ]:
cosine_graphic.data[0] = np.array([[-10, 0, 0]])

In [ ]:
# testing cell, ignore
plot_test("lines-data", fig_lines)

### Toggle the presence of a graphic within the scene

In [ ]:
sinc_graphic.present = False

In [ ]:
sinc_graphic.present = True

### You can create callbacks to this too, for example to re-scale the plot w.r.t. graphics that are present in the scene

In [ ]:
sinc_graphic.present.add_event_handler(subplot.auto_scale)

In [ ]:
sinc_graphic.present = False

In [ ]:
sinc_graphic.present = True

### You can set the z-positions of graphics to have them appear under other graphics

In [ ]:
img = iio.imread("imageio:camera.png")

subplot.add_image(img[::20, ::20], name="image", cmap="gray")

# z axis position -1 so it is below all the lines
subplot["image"].position_z = -1
subplot["image"].position_x = -8
subplot["image"].position_y = -8

In [ ]:
# testing cell, ignore
plot_test("lines-underlay", fig_lines)

In [ ]:
# close plot
fig_lines.close()

# Animation example with lines

In [ ]:
# just another example of animations
start, stop = 0, 2 * np.pi
increment = (2 * np.pi) / 50

# make a simple sine wave
xs = np.linspace(start, stop, 100)
ys = np.sin(xs)

fig = fpl.Figure()
fig[0, 0].add_line(ys, name="sine")

fig.show(maintain_aspect=False)

In [ ]:
# increment along the x-axis on each render loop :D 
def update_line(subplot):
    global increment, start, stop
    xs = np.linspace(start + increment, stop + increment, 100)
    ys = np.sin(xs)
    
    start += increment
    stop += increment

    # change only the y-axis values of the line
    subplot["sine"].data[:, 1] = ys


fig[0, 0].add_animations(update_line)

You can remove an animation

In [ ]:
fig[0, 0].remove_animation(update_line)

And add it back

In [ ]:
fig[0, 0].add_animations(update_line)

In [ ]:
fig.close()

### 3D line plot

In [ ]:
# just set the camera as "3d", the rest is basically the same :D
fig_l3d = fpl.Figure(cameras="3d")

# create a spiral
phi = np.linspace(0, 30, 200)

xs = phi * np.cos(phi)
ys = phi * np.sin(phi)
zs = phi

# use 3D data
# note: you usually mix 3D and 2D graphics on the same plot
spiral = np.column_stack([xs, ys, zs])

fig_l3d[0, 0].add_line(data=spiral, thickness=2, cmap='winter')

fig_l3d.show()

**Use WASD keys and the mouse to move around, just like in a game :D. Use the mouse weel to control the speed of movement.**

In [ ]:
fig_l3d[0, 0].auto_scale(maintain_aspect=True)

In [ ]:
# testing cell, ignore
plot_test("lines-3d", fig_l3d)

In [ ]:
# change the FOV of the persepctive camera
fig_l3d[0, 0].camera.fov = 70

In [ ]:
# change the controller, ex. from the current "fly" controller to a "panzoom" controller
fig_l3d[0, 0].controller = "panzoom"

In [ ]:
# or an orbit controller
fig_l3d[0, 0].controller = "orbit"

In [ ]:
# close plot
fig_l3d.close()

# A travelling electromagnetic wave :D 

In [ ]:
import fastplotlib as fpl
import numpy as np

In [ ]:
fig_em = fpl.Figure(
    cameras="3d", 
    controller_types="orbit", 
    size=(700, 400)
)

start, stop = 0, 4 * np.pi

# let's define the x, y and z axes for each with direction of wave propogation along the z-axis
# electric field in the xz plane travelling along
zs = np.linspace(start, stop, 200)
e_ys = np.zeros(200)
e_xs = np.sin(zs)
electric = np.column_stack([e_xs, e_ys, zs])

# magnetic field in the yz plane
zs = np.linspace(start, stop, 200)
m_ys = np.sin(zs)
m_xs = np.zeros(200)
magnetic = np.column_stack([m_xs, m_ys, zs])

# add the lines
fig_em[0, 0].add_line(electric, colors="blue", thickness=2, name="e")
fig_em[0, 0].add_line(magnetic, colors="red", thickness=2, name="m")

# draw vector line at every 10th position
electric_vectors = [np.array([[0, 0, z], [x, 0, z]]) for (x, z) in zip(e_xs[::10], zs[::10])]
magnetic_vectors = [np.array([[0, 0, z], [0, y, z]]) for (y, z) in zip(m_ys[::10], zs[::10])]

# add as a line collection
fig_em[0, 0].add_line_collection(electric_vectors, colors="blue", thickness=1.5, name="e-vec", z_offset=0)
fig_em[0, 0].add_line_collection(magnetic_vectors, colors="red", thickness=1.5, name="m-vec", z_offset=0)
# note that the z_offset in `add_line_collection` is not data-related
# it is the z-offset for where to place the *graphic*, by default with Orthographic cameras (i.e. 2D views)
# it will increment by 1 for each line in the collection, we want to disable this so set z_position=0

# axes are a WIP, just draw a white line along z for now
z_axis = np.array([[0, 0, 0], [0, 0, stop]])
fig_em[0, 0].add_line(z_axis, colors="w", thickness=1)

# just a pre-saved camera state
state = {
    'position': np.array([-8.0 ,  6.0, -2.0]),
    'rotation': np.array([0.09,  0.9 ,  0.2, -0.5]),
    'scale': np.array([1., 1., 1.]),
    'reference_up': np.array([0., 1., 0.]),
    'fov': 50.0,
    'width': 12,
    'height': 12,
    'zoom': 1.35,
    'maintain_aspect': True,
    'depth_range': None
}


fig_em[0, 0].camera.set_state(state)

fig_em.show()

In [ ]:
fig_em[0, 0].camera.zoom = 1.5

## Animation for the EM wave

In [ ]:
increment = np.pi * 4 / 100

# moves the wave one step along the z-axis
def tick(subplot):
    global increment, start, stop, zs
    new_zs = np.linspace(start, stop, 200)
    new_data = np.sin(new_zs)

    # just change the x-axis vals for the electric field
    subplot["e"].data[:, 0] = new_data
    # and y-axis vals for magnetic field
    subplot["m"].data[:, 1] = new_data

    # update the vector lines
    for i, (value, z) in enumerate(zip(new_data[::10], zs[::10])):
        subplot["e-vec"].graphics[i].data = np.array([[0, 0, z], [value, 0, z]])
        subplot["m-vec"].graphics[i].data = np.array([[0, 0, z], [0, value, z]])
    
    start += increment
    stop += increment

fig_em[0, 0].add_animations(tick)

# Scatter plots

### Plot tens of thousands or millions of points

#### There might be a small delay for a few seconds before the plot shows, this is due to shaders being compiled and a few other things. The plot should be very fast and responsive once it is displayed and future modifications should also be fast!

In [ ]:
# create a random distribution of 10,000 xyz coordinates
n_points = 10_000

# if you have a good GPU go for 1.5 million points :D 
# this is multiplied by 3
#n_points = 500_000

# dimensions always have to be [n_points, xyz]
dims = (n_points, 3)

clouds_offset = 15

# create some random clouds
normal = np.random.normal(size=dims, scale=5)
# stack the data into a single array
cloud = np.vstack(
    [
        normal - clouds_offset,
        normal,
        normal + clouds_offset,
    ]
)

# color each of them separately
colors = ["yellow"] * n_points + ["cyan"] * n_points + ["magenta"] * n_points

# create plot
fig_scatter = fpl.Figure()
subplot_scatter = fig_scatter[0, 0]
# use an alpha value since this will be a lot of points
scatter_graphic = subplot_scatter.add_scatter(data=cloud, sizes=3, colors=colors, alpha=0.7)

fig_scatter.show()

### scatter graphic features work similarly to line graphic

In [ ]:
# half of the first cloud's points to red
scatter_graphic.colors[:n_points:2] = "r"

In [ ]:
# set the green value directly
scatter_graphic.colors[n_points:n_points * 2, 1] = 0.3

In [ ]:
# set color values directly using an array
scatter_graphic.colors[n_points * 2:] = np.repeat([[1, 1, 0, 0.5]], n_points, axis=0)

In [ ]:
# change the data, change y-values
scatter_graphic.data[n_points:n_points * 2, 1] += 15

In [ ]:
# set x values directly but using an array
scatter_graphic.data[n_points:n_points * 2, 0] = np.linspace(-40, 0, n_points)

**Switch to a fly controller to move around the plot in 3D!**

In [ ]:
subplot_scatter.camera = "3d"
subplot_scatter.controller = "fly"

In [ ]:
# close plot
fig_scatter.close()

## More subplots

In [ ]:
# Figure of shape 2 x 3 with all controllers synced
figure_grid = fpl.Figure(shape=(2, 3), controller_ids="sync")

# Make a random image graphic for each subplot
for subplot in figure_grid:
    # create image data
    data = np.random.rand(512, 512)
    # add an image to the subplot
    subplot.add_image(data, name="rand-img")

# Define a function to update the image graphics with new data
# add_animations will pass the gridplot to the animation function
def update_data(f):
    for subplot in f:
        new_data = np.random.rand(512, 512)
        # index the image graphic by name and set the data
        subplot["rand-img"].data = new_data

# add the animation function
figure_grid.add_animations(update_data)

# show the gridplot
figure_grid.show()

### Slicing GridPlot

In [ ]:
# positional indexing
# row 0 and col 0
figure_grid[0, 0]

You can get the graphics within a subplot, just like with simple `Plot`

In [ ]:
figure_grid[0, 1].graphics

and change their properties

In [ ]:
figure_grid[0, 1].graphics[0].vmax = 0.5

more slicing with `GridPlot`

In [ ]:
# you can give subplots human-readable string names
figure_grid[0, 2].name = "top-right-plot"

In [ ]:
figure_grid["top-right-plot"]

In [ ]:
# view its position
figure_grid["top-right-plot"].position

In [ ]:
# these are really the same
figure_grid["top-right-plot"] is figure_grid[0, 2]

Indexing with subplot name and graphic name

In [ ]:
figure_grid["top-right-plot"]["rand-img"].vmin = 0.5

## Figure subplot customization

In [ ]:
# 2 rows and 3 columns
shape = (2, 3)

# pan-zoom controllers for each view
# views are synced if they have the
# same controller ID
controller_ids = [
    [0, 3, 1],  # id each controller with an integer
    [2, 2, 3]
]


# you can give string names for each subplot within the gridplot
names = [
    ["subplot0", "subplot1", "subplot2"],
    ["subplot3", "subplot4", "subplot5"]
]

# Create the grid plot
figure_grid = fpl.Figure(
    shape=shape,
    controller_ids=controller_ids,
    names=names,
)


# Make a random image graphic for each subplot
for subplot in figure_grid:
    data = np.random.rand(512, 512)
    # create and add an ImageGraphic
    subplot.add_image(data=data, name="rand-image")


# Define a function to update the image graphics
# with new randomly generated data
def set_random_frame(gp):
    for subplot in gp:
        new_data = np.random.rand(512, 512)
        subplot["rand-image"].data = new_data

# add the animation
figure_grid.add_animations(set_random_frame)
figure_grid.show()

Indexing the gridplot to access subplots

In [ ]:
# can access subplot by name
figure_grid["subplot0"]

In [ ]:
# can access subplot by index
figure_grid[0, 0]

**subplots also support indexing!**

this can be used to get graphics if they are named

In [ ]:
# can access graphic directly via name
figure_grid["subplot0"]["rand-image"]

In [ ]:
figure_grid["subplot0"]["rand-image"].vmin = 0.6
figure_grid["subplot0"]["rand-image"].vmax = 0.8

positional indexing also works event if subplots have string names

In [ ]:
figure_grid[1, 0]["rand-image"].vim = 0.1
figure_grid[1, 0]["rand-image"].vmax = 0.3

In [ ]:
# for testing, ignore
notebook_finished()